# Processing and joining Datasets into ``orders.csv``

This is our final stage with the creation of our dataset. We'll check what it has to be done
with orders.csv prior to joining data from other tables. Let's check our initial table!

In [1]:
import pandas as pd
import datetime
from pandas_profiling import ProfileReport

In [2]:
orders = pd.read_csv('./data/orders.csv')
orders

,order_id,lat,lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,2019-10-18 20:48:00+00:00,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,2019-10-19 01:00:00+00:00,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,2019-10-19 14:54:00+00:00,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,2019-10-18 21:47:00+00:00,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,2019-10-19 20:00:00+00:00,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822
...,...,...,...,...,...,...,...,...
9995,c63cfa783ff6ea9a5577ff0b513e4297,-33.417677,-70.508377,2019-10-19 01:00:00+00:00,False,9cf7b3c6a05745b24eb07e0945ff1718,c4ca4238a0b923820dcc509a6f75849b,116.631287
9996,0b91a6dc570736f490ade9b129bbd46f,-33.432885,-70.646795,2019-10-19 19:00:00+00:00,False,6fd0c09e971aff77c73afb87863da1c6,3871bd64012152bfb53fdf04b401193f,94.094497
9997,229d80f473668d9b25d60ea4d05687dd,-33.424562,-70.582943,2019-10-19 01:00:00+00:00,False,3ed51e3e0e8c9ca2fec3f84766cabe96,1f0e3dad99908345f7439f8ffabdffc4,53.784000
9998,96e2a7a69e52eaa5ce563b1164f72c8d,-33.413806,-70.598500,2019-10-19 18:00:00+00:00,False,cea50f5aa525cfe6d2f445fec7c8ef31,1f0e3dad99908345f7439f8ffabdffc4,52.806311


Before we join the tables, let's add the unix representation of promised_time,
we created a way of doing this on our 01 notebook.

EDIT: Unix representation is not as good, we can be more specific by
transforming date in two values: 'day' and 'hour'. We'll use only these two
values because from our unix time findings we have only a weekend's range
of data from Friday to Sunday in October 2019, otherwise we would
enrich the time features.

In [3]:
# create a function to map time to unix
def add_time_features(df):
    days = []
    hours = []
    for value in df['promised_time']:
        t_formatted = datetime.datetime.strptime(value, "%Y-%m-%d %H:%M:%S%z").timetuple()
        days.append(t_formatted[2])
        hours.append(t_formatted[3])
    df['day'] = days
    df['hour'] = hours

In [4]:
# add time features
add_time_features(orders)
# also delete time string
orders.drop('promised_time', axis=1, inplace=True)
orders

,order_id,lat,lng,on_demand,shopper_id,store_branch_id,total_minutes,day,hour
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,18,20
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,19,1
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN,19,14
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742,18,21
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822,19,20
...,...,...,...,...,...,...,...,...,...
9995,c63cfa783ff6ea9a5577ff0b513e4297,-33.417677,-70.508377,False,9cf7b3c6a05745b24eb07e0945ff1718,c4ca4238a0b923820dcc509a6f75849b,116.631287,19,1
9996,0b91a6dc570736f490ade9b129bbd46f,-33.432885,-70.646795,False,6fd0c09e971aff77c73afb87863da1c6,3871bd64012152bfb53fdf04b401193f,94.094497,19,19
9997,229d80f473668d9b25d60ea4d05687dd,-33.424562,-70.582943,False,3ed51e3e0e8c9ca2fec3f84766cabe96,1f0e3dad99908345f7439f8ffabdffc4,53.784000,19,1
9998,96e2a7a69e52eaa5ce563b1164f72c8d,-33.413806,-70.598500,False,cea50f5aa525cfe6d2f445fec7c8ef31,1f0e3dad99908345f7439f8ffabdffc4,52.806311,19,18


## Joining processed tables

### Adding ``order_products_processed.csv`` by ``order_id``

Let's load and join the table.

In [5]:
order_products = pd.read_csv('./data/order_products_processed.csv')
# TODO: eliminate this stupid residue index with Unnamed: 0, maybe loading lazy csv
order_products.drop('Unnamed: 0', axis=1, inplace=True)  # seems like we left residue
order_products


,sum_uni,sum_wei,pop_uni,pop_wei,order_id
0,44.0,12.700,3.121386,5.239230,47099653730fb1b76537fc10ad876255
1,34.0,2.300,-3.448815,4.034073,689d8866915acf87e851c2591a23a82f
2,35.0,0.240,-4.221307,1.475045,f26d16bf6f38c9e31d0be877f4013a9e
3,20.0,3.235,2.130173,5.720114,161ccc896835ab41761b0e726becb6b1
4,116.0,0.300,-8.079476,0.455584,4713deca10bb5db98fae150b52d61fc0
...,...,...,...,...,...
9973,1.0,0.000,-1.666715,0.000000,0ebd72696ae12532c87d9f3abcfb186e
9974,1.0,0.000,-1.262733,0.000000,6927e22f7f1a09ebbcd3e23877962aba
9975,1.0,0.000,-2.277916,0.000000,0ff3474dc3a636d3eb9dcc56ee3c7c8b
9976,1.0,0.000,-1.972315,0.000000,52a5da88b7693ea11a9ec6423c302726


In [6]:
# join tables
merged = orders.join(order_products.set_index('order_id'), on='order_id')
merged

,order_id,lat,lng,on_demand,shopper_id,store_branch_id,total_minutes,day,hour,sum_uni,sum_wei,pop_uni,pop_wei
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,18,20,16.0,2.756,-11.722202,5.802893
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,19,1,11.0,0.000,-2.337251,0.000000
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN,19,14,18.0,0.000,-5.956135,0.000000
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742,18,21,1.0,0.000,-1.262733,0.000000
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822,19,20,91.0,6.721,2.115241,10.121384
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,c63cfa783ff6ea9a5577ff0b513e4297,-33.417677,-70.508377,False,9cf7b3c6a05745b24eb07e0945ff1718,c4ca4238a0b923820dcc509a6f75849b,116.631287,19,1,116.0,16.634,3.946069,17.214263
9996,0b91a6dc570736f490ade9b129bbd46f,-33.432885,-70.646795,False,6fd0c09e971aff77c73afb87863da1c6,3871bd64012152bfb53fdf04b401193f,94.094497,19,19,24.0,0.600,-2.864198,-1.715620
9997,229d80f473668d9b25d60ea4d05687dd,-33.424562,-70.582943,False,3ed51e3e0e8c9ca2fec3f84766cabe96,1f0e3dad99908345f7439f8ffabdffc4,53.784000,19,1,16.0,3.502,4.382179,6.994387
9998,96e2a7a69e52eaa5ce563b1164f72c8d,-33.413806,-70.598500,False,cea50f5aa525cfe6d2f445fec7c8ef31,1f0e3dad99908345f7439f8ffabdffc4,52.806311,19,18,1.0,0.000,-1.793551,0.000000


### Adding ``shoppers_processed.csv`` by ``shopper_id``

Same method as the first table:

In [7]:
# load table
shoppers = pd.read_csv('./data/shoppers_processed.csv')
shoppers.drop('Unnamed: 0', axis=1, inplace=True)  # left residue again
shoppers

,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
0,1fc20b0bdf697ac13dd6a15cbd2fe60a,2,0.860600,1.94,1.000000,4.870000
1,e1c679ac73a69c01981fdd3c5ab8beda,0,0.844600,1.23,0.920000,4.920000
2,09d369c66ca86ebeffacb133410c5ee1,0,0.855900,1.56,1.000000,4.880000
3,db39866e62b95bb04ebb1e470f2d1347,1,0.861082,2.41,0.908276,4.848428
4,8efbc238660053b19f00ca431144fdae,0,0.877000,1.31,0.920000,4.880000
...,...,...,...,...,...,...
2859,da24da1311f7913f6d2d29d8238b439c,0,0.895100,1.53,0.880000,4.800000
2860,cf95eda5ffc1d4b9586de2ca08ab40f8,1,0.869500,3.00,0.560000,5.000000
2861,e8482e3ad8bc820ec756566a472b84b1,0,0.915200,1.47,0.880000,4.960000
2862,a55a3765a02530a97eb9af7aee327486,0,0.869500,1.20,0.960000,4.800000


In [8]:
# join by shopper id
merged = merged.join(shoppers.set_index('shopper_id'), on='shopper_id')
merged

,order_id,lat,lng,on_demand,shopper_id,store_branch_id,total_minutes,day,hour,sum_uni,sum_wei,pop_uni,pop_wei,seniority,found_rate,picking_speed,accepted_rate,rating
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,18,20,16.0,2.756,-11.722202,5.802893,0,0.9024,1.30,0.920000,4.76
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,19,1,11.0,0.000,-2.337251,0.000000,2,0.7610,2.54,0.920000,4.96
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN,19,14,18.0,0.000,-5.956135,0.000000,1,0.8313,2.57,0.760000,4.92
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742,18,21,1.0,0.000,-1.262733,0.000000,2,0.8776,2.80,0.960000,4.76
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822,19,20,91.0,6.721,2.115241,10.121384,1,0.7838,2.40,0.960000,4.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,c63cfa783ff6ea9a5577ff0b513e4297,-33.417677,-70.508377,False,9cf7b3c6a05745b24eb07e0945ff1718,c4ca4238a0b923820dcc509a6f75849b,116.631287,19,1,116.0,16.634,3.946069,17.214263,0,0.8658,1.30,0.960000,4.92
9996,0b91a6dc570736f490ade9b129bbd46f,-33.432885,-70.646795,False,6fd0c09e971aff77c73afb87863da1c6,3871bd64012152bfb53fdf04b401193f,94.094497,19,19,24.0,0.600,-2.864198,-1.715620,0,0.8970,1.41,1.000000,4.68
9997,229d80f473668d9b25d60ea4d05687dd,-33.424562,-70.582943,False,3ed51e3e0e8c9ca2fec3f84766cabe96,1f0e3dad99908345f7439f8ffabdffc4,53.784000,19,1,16.0,3.502,4.382179,6.994387,0,0.8424,1.40,1.000000,4.76
9998,96e2a7a69e52eaa5ce563b1164f72c8d,-33.413806,-70.598500,False,cea50f5aa525cfe6d2f445fec7c8ef31,1f0e3dad99908345f7439f8ffabdffc4,52.806311,19,18,1.0,0.000,-1.793551,0.000000,0,0.8529,1.98,0.933333,4.86


### Adding ``storebranch_processed.csv`` by ``store_branch_id``

Yep, same process:

In [9]:
# load the table
stores = pd.read_csv('./data/storebranch_processed.csv')
stores.drop('Unnamed: 0', axis=1, inplace=True)  # will fix next iteration I promise :)
# before we join, let's rename the lat, lng to lat_store, lng_store
stores.rename(columns={'lat': 'lat_store', 'lng': 'lng_store'}, inplace=True)
stores

,store_branch_id,lat_store,lng_store,branch_size,city
0,aff1621254f7c1be92f64550478c56e6,-33.422497,-70.609231,1,0
1,56352739f59643540a3a6e16985f62c7,-33.385484,-70.555579,3,0
2,7d04bbbe5494ae9d2f5a76aa1c00fa2f,-33.416579,-70.565224,23,0
3,2b24d495052a8ce66358eb576b8912c8,-33.512578,-70.655952,33,0
4,5487315b1286f907165907aa8fc96619,-33.347645,-70.542229,2,0
...,...,...,...,...,...
471,045117b0e0a11a242b9765e79cbf113f,-33.036111,-71.524149,17,3
472,2cfa3753d6a524711acb5fce38eeca1a,-33.406081,-70.598128,1,0
473,7486cef2522ee03547cfb970a404a874,-33.397498,-70.584666,1,0
474,c06d06da9666a219db15cf575aff2824,-33.390604,-70.547206,7,0


In [10]:
# join table by store_branch_id
merged = merged.join(stores.set_index('store_branch_id'), on='store_branch_id')
merged

,order_id,lat,lng,on_demand,shopper_id,store_branch_id,total_minutes,day,hour,sum_uni,...,pop_wei,seniority,found_rate,picking_speed,accepted_rate,rating,lat_store,lng_store,branch_size,city
0,e750294655c2c7c34d83cc3181c09de4,-33.501675,-70.579369,True,e63bc83a1a952fa2b3cc9d558fb943cf,65ded5353c5ee48d0b7d48c591b8f430,67.684264,18,20,16.0,...,5.802893,0,0.9024,1.30,0.920000,4.76,-33.485280,-70.579250,33,0
1,6581174846221cb6c467348e87f57641,-33.440584,-70.556283,False,195f9e9d84a4ba9033c4b6a756334d8b,45fbc6d3e05ebd93369ce542e8f2322d,57.060632,19,1,11.0,...,0.000000,2,0.7610,2.54,0.920000,4.96,-33.441246,-70.535450,33,0
2,3a226ea48debc0a7ae9950d5540f2f34,-32.987022,-71.544842,True,a5b9ddc0d82e61582fca19ad43dbaacb,07563a3fe3bbe7e3ba84431ad9d055af,NaN,19,14,18.0,...,0.000000,1,0.8313,2.57,0.760000,4.92,-33.008213,-71.545615,33,3
3,7d2ed03fe4966083e74b12694b1669d8,-33.328075,-70.512659,True,d0b3f6bf7e249e5ebb8d3129341773a2,f1748d6b0fd9d439f71450117eba2725,52.067742,18,21,1.0,...,0.000000,2,0.8776,2.80,0.960000,4.76,-33.355258,-70.537787,2,0
4,b4b2682d77118155fe4716300ccf7f39,-33.403239,-70.564020,False,5c5199ce02f7b77caa9c2590a39ad27d,1f0e3dad99908345f7439f8ffabdffc4,140.724822,19,20,91.0,...,10.121384,1,0.7838,2.40,0.960000,4.96,-33.386547,-70.568075,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,c63cfa783ff6ea9a5577ff0b513e4297,-33.417677,-70.508377,False,9cf7b3c6a05745b24eb07e0945ff1718,c4ca4238a0b923820dcc509a6f75849b,116.631287,19,1,116.0,...,17.214263,0,0.8658,1.30,0.960000,4.92,-33.402024,-70.516727,33,0
9996,0b91a6dc570736f490ade9b129bbd46f,-33.432885,-70.646795,False,6fd0c09e971aff77c73afb87863da1c6,3871bd64012152bfb53fdf04b401193f,94.094497,19,19,24.0,...,-1.715620,0,0.8970,1.41,1.000000,4.68,-33.451695,-70.692160,33,0
9997,229d80f473668d9b25d60ea4d05687dd,-33.424562,-70.582943,False,3ed51e3e0e8c9ca2fec3f84766cabe96,1f0e3dad99908345f7439f8ffabdffc4,53.784000,19,1,16.0,...,6.994387,0,0.8424,1.40,1.000000,4.76,-33.386547,-70.568075,33,0
9998,96e2a7a69e52eaa5ce563b1164f72c8d,-33.413806,-70.598500,False,cea50f5aa525cfe6d2f445fec7c8ef31,1f0e3dad99908345f7439f8ffabdffc4,52.806311,19,18,1.0,...,0.000000,0,0.8529,1.98,0.933333,4.86,-33.386547,-70.568075,33,0


### Eliminating residual id's

When we model, id values are not necessary so let's drop them!

In [11]:
merged.drop(['order_id', 'shopper_id', 'store_branch_id'], axis=1, inplace=True)
merged

,lat,lng,on_demand,total_minutes,day,hour,sum_uni,sum_wei,pop_uni,pop_wei,seniority,found_rate,picking_speed,accepted_rate,rating,lat_store,lng_store,branch_size,city
0,-33.501675,-70.579369,True,67.684264,18,20,16.0,2.756,-11.722202,5.802893,0,0.9024,1.30,0.920000,4.76,-33.485280,-70.579250,33,0
1,-33.440584,-70.556283,False,57.060632,19,1,11.0,0.000,-2.337251,0.000000,2,0.7610,2.54,0.920000,4.96,-33.441246,-70.535450,33,0
2,-32.987022,-71.544842,True,NaN,19,14,18.0,0.000,-5.956135,0.000000,1,0.8313,2.57,0.760000,4.92,-33.008213,-71.545615,33,3
3,-33.328075,-70.512659,True,52.067742,18,21,1.0,0.000,-1.262733,0.000000,2,0.8776,2.80,0.960000,4.76,-33.355258,-70.537787,2,0
4,-33.403239,-70.564020,False,140.724822,19,20,91.0,6.721,2.115241,10.121384,1,0.7838,2.40,0.960000,4.96,-33.386547,-70.568075,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-33.417677,-70.508377,False,116.631287,19,1,116.0,16.634,3.946069,17.214263,0,0.8658,1.30,0.960000,4.92,-33.402024,-70.516727,33,0
9996,-33.432885,-70.646795,False,94.094497,19,19,24.0,0.600,-2.864198,-1.715620,0,0.8970,1.41,1.000000,4.68,-33.451695,-70.692160,33,0
9997,-33.424562,-70.582943,False,53.784000,19,1,16.0,3.502,4.382179,6.994387,0,0.8424,1.40,1.000000,4.76,-33.386547,-70.568075,33,0
9998,-33.413806,-70.598500,False,52.806311,19,18,1.0,0.000,-1.793551,0.000000,0,0.8529,1.98,0.933333,4.86,-33.386547,-70.568075,33,0


In [12]:
# create a profile and save the dataset
# create a pandas profile for each table
def profile(df, name, minimal=True):
    prof = ProfileReport(df, minimal=minimal)
    prof.to_file(name)


profile(merged, './profiles/dataset.html')
merged.to_csv('./data/dataset.csv', index=False)

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]